In [1]:
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
data = np.load("data/processed_images.npy")
labels = np.load("data/labels_images.npy")
indices = np.random.choice(np.arange(len(data)), size=len(data), replace=False)
data = data[indices]
labels = labels[indices]
num_classes = len(set(labels))
labels = keras.utils.to_categorical(labels, num_classes)

print(data.shape)
print(labels.shape)

(60, 288, 352, 3)
(60, 8)


In [3]:
base_model = keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=[288,352,3], pooling=None, classes=num_classes)

In [4]:

# https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/
# Freeze the layers except the last 4 layers
for layer in base_model.layers[:-1]:
    layer.trainable = False


In [5]:
model = keras.Sequential()
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes, activation='softmax'))
# model.summary()

In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=1e-5),
              metrics=['acc'])
# Train the model
history = model.fit(x=data,y=labels,batch_size=1,epochs=60,validation_split=0.2)


Train on 48 samples, validate on 12 samples
Epoch 1/60


ResourceExhaustedError: OOM when allocating tensor with shape[202752,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_2/Adam/mul_2 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_2/Adam/sub_2, training_2/Adam/gradients/dense_1/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [9]:
base_model.layers[-1].trainable = True